In [2]:
import gzip
from collections import defaultdict

In [3]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [186]:
sum_of_rating = 0
u_i_r =[]
count = 0
for l in readGz("train.json.gz"):
   count += 1
   #if count<=100000 :
   sum_of_rating += l['rating']
   u_i_r.append([l['reviewerID'],l['itemID'],l['rating']])

In [187]:
train = u_i_r
validation = u_i_r[len(u_i_r)//2:]
alpha = sum_of_rating/200000
alpha_global = alpha

In [188]:
alpha_global

4.23471

In [8]:
### not for task, just try
import numpy as np
ttt = np.random.normal(size=(5,3),loc=0,scale=1)
ttt

array([[-0.08079462,  1.8011122 ,  0.38913561],
       [ 1.84796219,  0.24203708,  0.44194592],
       [ 0.32481568, -0.36336653,  1.33935022],
       [-0.05115383,  0.0168742 , -0.14008515],
       [ 0.41056996, -0.501754  , -0.77231606]])

In [189]:
## change!!! first train only in half!! then for the whole database
u_bought_r = defaultdict(list)
i_be_bought = defaultdict(list)
u_index = defaultdict(int)  # pair uID with a index
index_u = defaultdict(str)
u_count = 0                 # number of users
i_index = defaultdict(int)
index_i = defaultdict(str)
i_count = 0
for d in train :
    if d[0] not in u_index :
        u_index[d[0]] = u_count
        index_u[u_count]=d[0]
        u_count += 1
    if d[1] not in i_index :
        i_index[d[1]] = i_count
        index_i[i_count]=d[1]
        i_count += 1
   

In [191]:
for d in train :
     u = u_index[d[0]]
     i = i_index[d[1]]
     u_bought_r[u].append([i,d[2]])
     i_be_bought[i].append([u,d[2]])

In [192]:
for i in train :
    print(u_bought_r[u_index[i[0]]])
    print(i_be_bought[i_index[i[1]]])
    break
u_count

[[0, 4.0], [757, 3.0], [1679, 5.0]]
[[0, 4.0], [3487, 3.0], [10442, 3.0], [13847, 5.0], [15743, 1.0], [15287, 5.0], [13274, 5.0], [5895, 5.0], [29634, 4.0], [6132, 3.0], [17049, 5.0], [13201, 5.0], [17412, 3.0], [34277, 5.0], [12148, 5.0], [868, 5.0], [36249, 4.0], [16660, 5.0], [38495, 4.0]]


39239

In [99]:
## see sgd1, this one is useless , 

def sgd(train,step,lamda, k):
    beta_u = np.zeros(u_count)
    beta_i = np.zeros(i_count)
    gamma_u = np.random.normal(size=(u_count,k),scale=1/k) # matrix : num of u * k
    gamma_i = np.random.normal(size=(i_count,k),scale=1/k)
    alpha = alpha_global
    for i in range(150):
       # change of alpha
       gd = 0
       for l in train:
          u = u_index[l[0]]
          i = i_index[l[1]]
          gd += 2*(alpha+beta_u[u]+beta_i[i]\
                 +np.dot(gamma_u[u],gamma_i[i])-l[2])
       alpha -= gd*step /len(train)
       print(alpha)
    
       # change of beta_u & gamma_u
       for u in u_bought_r :
          gd = 0
          gm = np.zeros(k)
          for i in u_bought_r[u]:
             temp = 2*(alpha+beta_u[u]+beta_i[i[0]]+np.dot(gamma_u[u],gamma_i[i[0]])-i[1])
             gd += temp
             gm += temp*gamma_i[i[0],:]
          
          gd += 2*lamda*beta_u[u]
          gd /= len(u_bought_r)
          beta_u[u] -= gd*step
          gm += 2*lamda*gamma_u[u,:]
          gm /= len(u_bought_r)
          gamma_u[u,:] -= gm*step
         # self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
       print("beta_u[0] "+ str(beta_u[0]))
      
       # change of beta_i & gamma_i
       for i in i_be_bought :
          gd = 0
          gm = np.zeros(k)
          for u in i_be_bought[i]:
              temp = 2*(alpha+beta_u[u[0]]+beta_i[i]+np.dot(gamma_u[u[0]],gamma_i[i])-u[1])
              gd += temp
              gm += temp*gamma_u[u[0],:]
                
          gd += 2*lamda*beta_i[i]
          gd /= len(i_be_bought)
          beta_i -= gd*step
          gm += 2*lamda*gamma_i[i,:]
          gm /= len(i_be_bought)
          gamma_i[i,:] -= gm*step
        
    return alpha, beta_u,beta_i,gamma_u,gamma_i

In [101]:
err = 0
p_rating = 0
for i in validation:
   p_rating = a + bu[u_index[i[0]]]+ bi[i_index[i[1]]]+np.dot(gu[u_index[i[0]]],gi[i_index[i[1]]])
   err += (p_rating - i[2])**2
 
mse1 = err/len(validation)
print("the MSE of the validation set is " + str(mse1))

the MSE of the validation set is 2.144317996744941


In [194]:
def sgd1(k,step,lamda):
    beta_u = np.zeros(u_count)
    beta_i = np.zeros(i_count)
    gamma_u = np.random.normal(size=(u_count,k),scale=1/k) # matrix : num of u * k
    gamma_i = np.random.normal(size=(i_count,k),scale=1/k)
    alpha = alpha_global
    
    for i in range(30):
        for (u,i,r) in train:
           u_inx = u_index[u]
           i_inx = i_index[i]
           prediction = alpha+beta_u[u_inx]+beta_i[i_inx]+np.dot(gamma_u[u_inx],gamma_i[i_inx])
           e = r - prediction
            # alpha?
        #   alpha += step*2*e
        
           beta_u[u_inx] += step*(2*e-2*lamda*beta_u[u_inx])
           beta_i[i_inx] += step*(2*e-2*lamda*beta_i[i_inx])
            
           gamma_u[u_inx,:] += step*(2*e*gamma_i[i_inx,:]-2*lamda*gamma_u[u_inx,:])
       #    print(gamma_u[u_inx,:])
           gamma_i[i_inx,:] += step*(2*e*gamma_u[u_inx,:]-2*lamda*gamma_i[i_inx,:])
      #     print(gamma_i[i_inx,:])
        err = 0
        p_rating = 0
        for i in validation:
           p_rating = alpha + beta_u[u_index[i[0]]]+ beta_i[i_index[i[1]]]+np.dot(gamma_u[u_index[i[0]]],gamma_i[i_index[i[1]]])
           err += (p_rating - i[2])**2
        mse1 = err/len(validation)
        print("the MSE of the validation set is " + str(mse1)) 
    return beta_u,beta_i,gamma_u,gamma_i,alpha

In [200]:
beta_u,beta_i,gamma_u,gamma_i,alpha=sgd1(1,0.01,1)

the MSE of the validation set is 1.2413417614220725
the MSE of the validation set is 1.0318129052730458
the MSE of the validation set is 0.9407638974760305
the MSE of the validation set is 0.8911834067822355
the MSE of the validation set is 0.8607752587445859
the MSE of the validation set is 0.8407269681472669
the MSE of the validation set is 0.8268664387515715
the MSE of the validation set is 0.8169651297944449
the MSE of the validation set is 0.8097252293655723
the MSE of the validation set is 0.8043407440936892
the MSE of the validation set is 0.8002859461717617
the MSE of the validation set is 0.7972045902634068
the MSE of the validation set is 0.7948479049419351
the MSE of the validation set is 0.7930379397230916
the MSE of the validation set is 0.7916448983765586
the MSE of the validation set is 0.7905725825318386
the MSE of the validation set is 0.7897487427038954
the MSE of the validation set is 0.7891185089870641
the MSE of the validation set is 0.7886398159638194
the MSE of t

In [201]:
predictions_rating = open("predictions_rating_Ass_1.txt", 'w')
for l in open("pairs_rating.txt"):
   if l.startswith("reviewerID"):
   #header
      predictions_rating.write(l)
      continue
   u,i = l.strip().split('-')
   u_pos = u_index[u]
   i_pos = i_index[i]
   rting = alpha + beta_u[u_pos] +beta_i[i_pos]+np.dot(gamma_u[u_pos],gamma_i[i_pos])
   print(rting)
   predictions_rating.write(u + '-' + i + ',' + str(rting) + '\n')
predictions_rating.close()

4.1522573127978255
4.272561574037133
4.052060850397983
4.194781592530227
4.257934473899155
4.677836905041222
4.443505408802652
4.514794871497558
4.661631407242497
3.3731219065431692
4.199463570441875
3.710515981616889
4.719568833372487
4.144803987759634
4.387983553466995
4.490281227552576
4.771701733439446
4.3170634751108015
4.017694844851203
2.7668224102761485
3.6873910826769527
3.769796205512514
3.706468041615508
3.6108553118164
4.437599097564753
4.745643577320195
4.526821759799336
3.901984676440451
3.513277576603269
4.580207589918735
4.3651314172008675
4.422918009869858
4.0169277437117055
3.384701478371059
4.949135706236285
4.009178520314841
4.140613062211547
4.35561181571965
4.654434177195353
4.261298894869479
4.491504807975486
4.631456235326734
3.8730530549250424
4.266172134279009
4.3482458196080325
4.561393391037174
4.0352470003775
4.792138552436095
3.800551311078304
4.106050491683783
4.2962843000076205
4.67207737663408
4.031230257108855
4.693587956821536
4.440182307051415
4.2871

4.01655823604617
4.629106083683492
4.868388819201761
3.879221453716241
4.29440088556052
3.834967287291069
4.561861643039099
4.78628133735622
4.35597772723145
4.497949308125686
4.419195562232315
4.3505488619657005
4.524810664117453
3.735703319193322
4.610176715696609
4.644223816517466
4.193413257227776
4.738547916297355
4.67211924653263
4.10732624233628
4.367568055137397
4.502557318967316
3.913763244183995
3.9730266316629486
4.067895803774565
3.33279852462814
4.670668458494705
4.035541602212273
4.528944872731698
4.1794790495061145
4.490650292091107
4.013580347562859
3.58367299546012
4.200394081303672
4.5769059971246335
4.70284063280475
4.0910285771582675
4.004676490479312
4.233442221848591
4.372812372102541
4.411236630210542
4.130831845901415
4.27651118619024
4.483884196385989
4.679655970858406
4.360880544815585
3.9687694595378336
4.251678913577499
4.705760647013359
3.535782971522046
3.626728621850987
4.457752311254978
4.711345184420713
5.002122379150082
4.329530605658884
3.552964628020

4.233749724636393
4.600351285184859
4.837714489521406
3.6467809769962733
4.029419926097567
4.445821839429565
3.593224179459477
4.370555727298128
3.872225655706349
3.679959007255474
4.111427495876041
4.295117192956528
3.947588859642824
4.351626522009964
3.8068292221145463
4.487993613055478
4.6413297694177515
4.691035599262477
3.7473238732181064
3.9070810740485564
4.488592629552942
3.8248439404915713
4.184975061715979
4.397438672011589
4.55897989000541
4.316106978381146
4.474553732690395
3.5944257133699153
3.5725642885952533
4.402897219355655
3.979824462911762
4.743933404659332
4.219913279212338
3.558067234133799
4.758883424989726
4.229526783293748
4.556308511728508
4.535277618616972
4.563483274261523
3.6248684739965276
4.391470396402271
4.3754873262890115
4.666441765860895
4.553923646905421
3.788988407405704
4.655157235796539
4.692331825123044
3.9967080695792174
3.97716818162102
4.445704435831108
4.422388941418128
3.708899292478825
3.768195382016128
3.447298946922767
4.145383769474843
4

4.344783223764088
4.7508027323211355
4.660716989218075
4.392621161861554
3.642279217679937
4.657383010016167
3.787003346609085
4.6815147960879875
4.7982059561852415
4.693357675316972
4.52529173689913
3.6828700018970344
4.289737580679423
4.601345227040033
4.121309948023168
4.395502054258216
4.065317629062175
4.334016112667952
4.858638236260492
3.4665529688153645
4.571645608323638
4.00356534873189
3.946868863970866
3.804727436355841
4.459104085268467
4.520655416393196
4.7674827435206915
4.3147328824541304
4.136685186963874
4.417449675143633
4.580624239718227
4.361972469323982
4.12524676085946
3.914139600059259
4.115361878559554
3.6520618657701243
4.71136521061208
4.610588668148714
4.301592066563658
4.3309825629801715
4.5361725435043505
4.139915906078014
4.2857440343614215
4.10323578942095
3.8522682313294285
4.303325924042809
4.010156909775629
4.301158365048723
4.229248968224767
4.318732736373003
4.528365951740044
4.624010731369444
4.3570255078108895
4.243126827905968
4.522723812224478
4.

4.349787139759096
3.880380184479137
3.542414119537071
4.535121321366644
4.262835808907672
3.783600283819005
4.3085581598863945
3.786509383665402
3.9362328875595147
3.986216014763828
3.7229699450334013
4.092430339788857
4.42370949637259
4.354578683712035
4.165789594558218
3.60362717918791
4.028701552882213
4.53838689241038
3.2494053659541473
4.752922726536282
4.098996611341624
4.586234643879251
4.704336427096463
3.469920444631008
3.9669294664612345
2.7103870033139494
4.546583447169637
4.107313039764431
4.179110149928392
4.097030485661134
4.092093270662653
4.59306199777193
4.296380949953216
3.6767242842245706
3.7400094583161634
3.862405167484122
4.515511709994434
4.016125047586828
3.959210447262555
4.115099421256015
4.472214293016083
4.349666611985888
4.146641646518352
3.8143503868613466
4.153024100202648
3.9140268379289114
3.9103261430223024
4.698293309711292
4.111994367359508
4.3051143450592
3.2349815415846255
4.225198753922788
4.568322191385098
4.566916289572065
4.29772641276927
4.148

3.4271983702299003
3.516155831139379
3.803795295332352
4.739491788729292
4.743142232209023
4.312970268125884
4.5642912895032355
4.146883782662685
4.838139437058665
4.768317680312074
3.76735530599061
4.5847454759232456
4.348746609104375
4.500703810095406
4.436987277514234
3.758335965042081
4.215390436431087
4.090606994462392
3.763169638843402
4.478308890092093
4.353218205276041
4.61190599528097
4.163799100725368
3.69043983429217
4.1205140909589355
3.6238366196527245
4.441625284902881
4.203424180944496
3.421107848441783
4.583717649567057
4.4581757925481424
4.522355693893714
4.482713734379746
4.185345085838269
4.304616445600449
3.9307447910664557
4.606262138526729
4.249992006894117
3.5207522947195686
4.249234809976811
3.795586081999827
4.245812130608869
4.411902472398836
4.571833763942493
3.324186907621706
4.391192017322496
4.710061638065952
4.103905570316209
4.206176674799456
4.025037045379136
3.80366254551188
4.101084698279743
4.529449265437801
3.890210096510457
3.5620906223131032
3.608

4.06318889821698
3.5603825085715983
4.724883863621503
4.130109120864065
4.198193155823086
4.604972561705333
4.162352981604691
3.9950856843516944
4.471790966347773
4.712410247230391
4.5015941536756205
4.179163463378402
3.644545310283701
3.1891178856896283
4.32591847643368
3.9869996579492697
4.012417466238652
4.367134629917477
4.343850218738222
4.847276924677128
4.317454346125512
3.8931594913965526
3.8963744728301246
4.424775151001261
4.3248026130299735
4.684449646191956
4.5852176236127855
3.6489282803934424
4.019452872832424
4.1655076620498
4.775923701827894
3.4718135686766853
4.196621522627766
4.124829502494183
3.773592354717938
4.687981717937014
3.7001543968303228
3.999775473297796
4.160102340975663
4.772922498581812
3.278109936244514
4.374669991998803
4.550454029692592
4.560823385512549
3.5226096241469524
3.749120921556834
4.132500153189357
4.37817146748241
4.133502249398141
4.002009012367087
4.544778197008698
3.732496583106838
4.276115641892859
4.157703564604723
4.749531552739346
4.